## Dependencies 

In [108]:
import os
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [109]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\rafa_\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


## Scraping

   #### Mars News

In [64]:
#URL used to scrape
url = 'https://mars.nasa.gov/news/'

browser.visit(url)

In [65]:
html = browser.html
n_soup = bs(html, 'html.parser')

In [69]:
#Find the title of the first news article
title_elem = n_soup.select_one('ul.item_list li.slide')
title_elem = title_elem.find('div', class_='content_title').get_text()
title_elem

'Independent Review Indicates NASA Prepared for Mars Sample Return Campaign'

In [71]:
#Get the paragraph for the news article found
paragraph_elem = n_soup.select_one('ul.item_list li.slide')
paragraph_elem = paragraph_elem.find('div', class_='article_teaser_body').get_text()
paragraph_elem

'NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.'

#### JPL Mars Space Images

In [94]:
#URL used to scrape
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [95]:
#Find the image called full_image
featured_image_url = browser.find_by_id("full_image")
featured_image_url.click()

In [96]:
#Go through the More Info button
moreinfo = browser.links.find_by_partial_text('more info')
moreinfo.click()

In [97]:
html = browser.html
img_soup = bs(html,'html.parser')

In [98]:
#Assign the url
img_url = img_soup.find('figure',class_='lede').find('a').get('href')

In [99]:
img_url

'/spaceimages/images/largesize/PIA17011_hires.jpg'

In [100]:
featured_url = 'https://www.jpl.nasa.gov' + img_url
featured_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17011_hires.jpg'

#### Mars Facts

In [101]:
#URL used to scrape
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [104]:
table = pd.read_html(url)
table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [106]:
#Make it into a pandas column
fact_table = table[0]
fact_table.columns = ["Category", "Unit"]
fact_table

,Category,Unit
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [112]:
#Convert it back to HTML
fact_html = fact_table.to_html()
fact_html.replace('\n','')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Category</th>      <th>Unit</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers<

#### Mars Hemispheres

In [124]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [125]:
html = browser.html
hemispheres_soup = bs(html, 'html.parser')

In [126]:
hemispheres_url = hemispheres_soup.find_all('div', class_='item')

In [129]:
hem_dict = []

for hem in range(len(hemispheres_url)):
    hem_item = {}    
    
    # click on each of the links
    browser.find_by_css('a.product-item h3')[hem].click()
    
    # get the enhanced image title
    hem_item["img_title"] = browser.find_by_css('h2.title').text
    # get the enhanced image link
    hem_item["img_url"] = browser.links.find_by_text('Sample')['href']
        
    # add it to the hemisphere dictionary
    hem_dict.append(hem_item)
    
    # have to go back to the main browser page
    browser.back()

print(hem_dict)

[{'img_title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'img_title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'img_title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'img_title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [130]:
browser.quit()